In [ ]:
# ============================================================
# ✅ System Environment Summary (Colab / Local / HPC 공용)
# ============================================================
import platform, os, sys, subprocess, torch, psutil, datetime

def get_env_info():
    print("=== SYSTEM ENVIRONMENT REPORT ===")
    print(f"[Timestamp] {datetime.datetime.now().isoformat()}")
    print(f"[Python] {sys.version}")
    print(f"[Platform] {platform.system()} {platform.release()} ({platform.machine()})")
    print(f"[Processor] {platform.processor()}")
    print(f"[Python Executable] {sys.executable}")
    print(f"[Working Dir] {os.getcwd()}")
    print(f"[PID] {os.getpid()} | User: {os.getenv('USER')}")
    print(f"[CUDA Available] {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  cuDNN Version: {torch.backends.cudnn.version()}")
        print(f"  GPU Count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"   └─ GPU {i}: {torch.cuda.get_device_name(i)} "
                  f"({torch.cuda.get_device_properties(i).total_memory/1024**3:.2f} GB)")
    else:
        print("  No GPU detected.")

    # CPU/MEM
    mem = psutil.virtual_memory()
    print(f"[CPU] {psutil.cpu_count(logical=False)} cores ({psutil.cpu_count()} threads)")
    print(f"[Memory] {mem.total/1024**3:.2f} GB total | {mem.available/1024**3:.2f} GB free")
    print(f"[Disk] {psutil.disk_usage('/').total/1024**3:.2f} GB total | "
          f"{psutil.disk_usage('/').free/1024**3:.2f} GB free")

    # Important library versions
    def ver(name):
        try:
            return subprocess.check_output([sys.executable, "-m", "pip", "show", name]).decode().split("Version: ")[1].split("\n")[0]
        except Exception:
            return "N/A"
    libs = ["numpy", "scipy", "torch", "pandas", "matplotlib"]
    print("[Packages]")
    for lib in libs:
        print(f"  {lib:<10s} {ver(lib)}")

    # Python path & env
    print(f"[Sys Path] {sys.path[:3]} ...")
    print(f"[Env Vars] LANG={os.getenv('LANG')} | PATH snippet={os.getenv('PATH')[:80]}...")
    print("=== END ENV REPORT ===\n")

get_env_info()


=== SYSTEM ENVIRONMENT REPORT ===
[Timestamp] 2025-10-12T17:33:12.541096
[Python] 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
[Platform] Linux 6.6.97+ (x86_64)
[Processor] x86_64
[Python Executable] /usr/bin/python3
[Working Dir] /content
[PID] 6350 | User: None
[CUDA Available] False
  No GPU detected.
[CPU] 4 cores (8 threads)
[Memory] 50.99 GB total | 49.39 GB free
[Disk] 225.83 GB total | 186.41 GB free
[Packages]
  numpy      2.0.2
  scipy      1.16.2
  torch      2.8.0+cu126
  pandas     2.2.2
  matplotlib 3.10.0
[Sys Path] ['/content', '/env/python', '/usr/lib/python312.zip'] ...
[Env Vars] LANG=en_US.UTF-8 | PATH snippet=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bi...
=== END ENV REPORT ===



In [ ]:
# -*- coding: utf-8 -*-
# ============================================================
# BFGS/L-BFGS-B/Adam/AdamW 비교 + 지형 메트릭 로깅 + 결정공간 슬라이스
# - 매 스텝 CSV 로깅(phase 포함)
# - 3개 물리 단면(YZ/XZ/XY) + 최종 1D/2D 결정공간 슬라이스 저장
# - (간헐) Lanczos로 H의 극값 고유치 추정
# - 컬러바 스케일: XY(z=z0) 단면의 max를 YZ/XZ/XY 공통 vmax로 사용
# - 메서드 선택(input):
#     conventional_twin | conventional_vortex
#   | regularized_conventional_twin | regularized_conventional_vortex
#   | force-equillibrium | hybrid_twin | hybrid_vortex
#   | regularized_hybrid_twin | regularized_hybrid_vortex
# - Weighted Laplacian: U_xx, U_yy, U_zz 축별 가중치 적용
# ============================================================

import os
import time
import math
import numpy as np
import pandas as pd

# Headless backend (no X server)
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib import colors

import torch
from scipy.optimize import minimize

# ============================================================
# [사용자 편집 구역] (기본값; 실제 사용값은 configure_method()에서 메서드에 따라 덮어씀)
# ============================================================
CSV_FILENAME = "optimization_steps_data_with_phases_and_terrain.csv" # per-step 로그 CSV
BASE_OUTPUT_DIR = os.path.join("results_github") # 메서드별 하위 폴더 생성
RUN_TAG = "BFGS_8x8_rand42_fixed_10000_steps" # 공통 러닝 태그

# 트래핑 목표 지점 (미터 좌표)
TARGET_COORD = (0.01, 0.01, 0.03) # (x, y, z) in meters

# 공통 가중치 심볼(목표함수 조합에 사용)
#  maximize [ w_lap*Δ²_w U  -  α*|∇U|  -  β*Penalty(|p|) ]
#  -> loss = -metric
W_LAPLACIAN = 1.0 # Δ²_w U 전체 계수  (메서드별 고정 1.0)
ALPHA_F = 500.0   # |∇U| 계수 (Force-Equilibrium/Hybrid에서 사용)
BETA_P = 5e-5     # |p| 계수 (메서드에 따라 1.0 또는 5e-5)

# Weighted Laplacian (각 방향별 가중치; 메서드에 따라 설정)
# Δ²_w U = W_XX * U_xx + W_YY * U_yy + W_ZZ * U_zz
W_LAPLACIAN_XX = 1
W_LAPLACIAN_YY = 1
W_LAPLACIAN_ZZ = 1

# Charbonnier epsilon 상대값 (로컬 RMS 기준; regularized* 에서 사용)
EPS_REL = 1e-3 # 섹션 2.2.4의 0.001 규칙

# 결과 플롯: 압력장 3개 단면(YZ@x*, XZ@y*, XY@z*)
GENERATE_SLICE_PLOTS = True

# 결정변수 공간 지형 시각화(최종점 기준)
GENERATE_DECISION_SLICES = True
SLICE_HALF_RANGE = 0.30 # 각 축 ±범위 [rad]
SLICE_N_1D = 81 # 1D 샘플 개수
SLICE_N_2D = 61 # 2D 격자 한 변 샘플 개수

# 최적화(실험 반복) 설정
NUM_TRIALS = 100
NUM_OPT_STEPS = 10000

# 헤시안 분광 추정 간격/정밀도 (매 스텝 수행은 과중하므로 간헐적)
SPEC_EVERY = 200 # 200 스텝마다 시도
SPEC_K = 10 # Lanczos 반복 수(작을수록 가벼움)
HVP_EPS = 1e-4 # 유한차분 H·v epsilon
# ============================================================

# ============================================================
# 환경/출력 설정 (메서드 선택은 맨 아래에서 처리)
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(random_seed)
print(f"[INFO] Using Fixed Random Seed: {random_seed}")

# ============================================================
# 물리/격자 파라미터
# ============================================================
amp_table = torch.tensor([
    10.000, 11.832, 13.509, 15.414, 17.615, 18.240, 20.172, 21.626, 22.309, 23.054, 23.820, 23.820, 23.820,
    24.650, 24.650, 26.306, 29.052, 32.140, 35.496, 39.243, 41.952, 46.411, 47.958, 49.598, 53.009, 56.657,
    58.652, 60.581, 62.610, 64.885, 67.007, 69.282, 71.624, 71.624, 76.551, 76.551, 79.183, 84.676, 84.676,
    87.579, 90.388, 93.488, 93.488, 93.488, 96.695, 100.000, 96.695, 93.488, 93.488, 93.488, 90.388, 87.579,
    84.676, 84.676, 79.183, 76.551, 76.551, 71.624, 71.624, 69.282, 67.007, 64.885, 62.610, 60.581, 58.652,
    56.657, 53.009, 49.598, 47.958, 46.411, 41.952, 39.243, 35.496, 32.140, 29.052, 26.306, 24.650, 24.650,
    23.820, 23.820, 23.820, 23.054, 22.309, 21.626, 20.172, 18.240, 17.615, 15.414, 13.509, 11.832, 10.000
], dtype=torch.float64, device=device)

frequency = 40000.0 # Hz
speed_of_sound = 343.0 # m/s
wavelength = speed_of_sound / frequency

# 8x8 배열 (1 cm pitch)
x_coords = np.arange(-3.5, 4.5, 1.0)
y_coords = np.arange(-3.5, 4.5, 1.0)
transducer_positions = torch.tensor(
    [(x * 0.01, y * 0.01, 0.0) for x in x_coords for y in y_coords],
    dtype=torch.float64, device=device
)
num_transducers = len(transducer_positions)
print(f"[INFO] Transducer grid: 8x8 ({num_transducers} transducers).")

# 관측 격자 (201^3)
grid_size = 201
x_vals = torch.linspace(-0.05, 0.05, grid_size, dtype=torch.float64, device=device)
y_vals = torch.linspace(-0.05, 0.05, grid_size, dtype=torch.float64, device=device)
z_vals = torch.linspace(0.0, 0.1, grid_size, dtype=torch.float64, device=device)
dx = (x_vals[1] - x_vals[0]).item()
dy = (y_vals[1] - y_vals[0]).item()
dz = (z_vals[1] - z_vals[0]).item()

# 목표 지점 (격자 인덱스)
target_coord_x, target_coord_y, target_coord_z = TARGET_COORD
x_idx = torch.argmin(torch.abs(x_vals - target_coord_x)).item()
y_idx = torch.argmin(torch.abs(y_vals - target_coord_y)).item()
z_idx = torch.argmin(torch.abs(z_vals - target_coord_z)).item()
target_x = x_vals[x_idx].item()
target_y = y_vals[y_idx].item()
target_z = z_vals[z_idx].item()
print(f"[INFO] Target Index: ({x_idx}, {y_idx}, {z_idx}) | Target Coord: ({target_x:.4f}, {target_y:.4f}, {target_z:.4f})")

# 로컬 윈도우(5x5x5) 반경
half = 2 # 2 -> (2*2+1)=5 포인트

# ============================================================
# 메서드-의존 파라미터 (런타임에 세팅)
# ============================================================
METHOD_NAME = None
PRESSURE_PENALTY_MODE = 'abs' # 'abs' or 'smooth_abs'
CURRENT_WEIGHTS = dict(w_lap=W_LAPLACIAN, alpha=0.0, beta=0.0)
OUTPUT_DIR = None

# ============================================================
# 계산 함수
# ============================================================
def compute_pressure_field_torch(phase_vector, amp_table, wavelength, X, Y, Z, transducer_positions):
    k_val = 2.0 * math.pi / wavelength
    grid_points = torch.stack([X.reshape(-1), Y.reshape(-1), Z.reshape(-1)], dim=1) # [M,3]
    delta = grid_points.unsqueeze(1) - transducer_positions.unsqueeze(0) # [M,N,3]
    R = torch.linalg.norm(delta, dim=2).clamp_min(1e-9)
    cos_theta = torch.clamp(delta[:, :, 2] / R, -1.0, 1.0)
    theta_deg = torch.rad2deg(torch.acos(cos_theta)).clamp(0.0, 90.0)

    max_idx = amp_table.shape[0] - 1
    low_index_f = theta_deg # 1 deg grid
    low_index = torch.floor(low_index_f).long()
    high_index = torch.clamp(low_index + 1, max=max_idx)
    frac = (low_index_f - low_index.to(torch.float64))
    A_low = amp_table[low_index]
    A_high = amp_table[high_index]
    A_theta = A_low + frac * (A_high - A_low)

    amplitude = A_theta / R
    propagation_phase = k_val * R
    total_phase = phase_vector.unsqueeze(0) + propagation_phase # [M,N]
    p_complex = torch.polar(amplitude, total_phase) # complex128
    p_field = torch.sum(p_complex, dim=1) # [M]
    return p_field.reshape(X.shape)

def compute_gradient_torch(U, dx, dy, dz):
    return torch.gradient(U, spacing=(dx, dy, dz), edge_order=1)

def compute_laplacian_weighted_torch(U, dx, dy, dz):
    grad_U_x, grad_U_y, grad_U_z = compute_gradient_torch(U, dx, dy, dz)
    L_xx, _, _ = torch.gradient(grad_U_x, spacing=(dx, dy, dz), edge_order=1)
    _, L_yy, _ = torch.gradient(grad_U_y, spacing=(dx, dy, dz), edge_order=1)
    _, _, L_zz = torch.gradient(grad_U_z, spacing=(dx, dy, dz), edge_order=1)
    return (W_LAPLACIAN_XX * L_xx +
            W_LAPLACIAN_YY * L_yy +
            W_LAPLACIAN_ZZ * L_zz)

def compute_laplacian_unweighted_torch(U, dx, dy, dz):
    grad_U_x, grad_U_y, grad_U_z = compute_gradient_torch(U, dx, dy, dz)
    L_xx, _, _ = torch.gradient(grad_U_x, spacing=(dx, dy, dz), edge_order=1)
    _, L_yy, _ = torch.gradient(grad_U_y, spacing=(dx, dy, dz), edge_order=1)
    _, _, L_zz = torch.gradient(grad_U_z, spacing=(dx, dy, dz), edge_order=1)
    return L_xx + L_yy + L_zz

def pressure_penalty(p2_center, p_abs_center, p2_local_rms):
    if PRESSURE_PENALTY_MODE == 'smooth_abs':
        eps = EPS_REL * (p2_local_rms + 1e-32)
        return torch.sqrt(p2_center + eps*eps)
    elif PRESSURE_PENALTY_MODE == 'abs':
        return torch.sqrt(p2_center + 1e-32)
    else:
        raise ValueError(f"Unknown PRESSURE_PENALTY_MODE: {PRESSURE_PENALTY_MODE}")

def compute_gorkov_objective_local_torch(pf, dx, dy, dz):
    rho0, c0, rho_p, c_p = 1.225, 343.0, 100.0, 2400.0
    omega = 2 * math.pi * frequency
    r = 1.3e-3 / 2
    V = 4/3 * math.pi * r**3

    K1 = 0.25 * V * (1 / (c0**2 * rho0) - 1 / (c_p**2 * rho_p))
    K2 = 0.75 * V * ((rho0 - rho_p) / (omega**2 * rho0 * (rho0 + 2 * rho_p)))

    abs_p2 = (pf.real**2 + pf.imag**2)
    dpdx, dpdy, dpdz = compute_gradient_torch(pf, dx, dy, dz)
    v_sq = (dpdx.real**2 + dpdx.imag**2) + (dpdy.real**2 + dpdy.imag**2) + (dpdz.real**2 + dpdz.imag**2)

    U = K1 * abs_p2 - K2 * v_sq
    gradUx, gradUy, gradUz = compute_gradient_torch(U, dx, dy, dz)

    lapU_weighted = compute_laplacian_weighted_torch(U, dx, dy, dz)
    lapU_unweighted = compute_laplacian_unweighted_torch(U, dx, dy, dz)

    center_idx = (half, half, half)
    laplacian_center_weighted = lapU_weighted[center_idx]
    laplacian_center_unweighted = lapU_unweighted[center_idx]

    grad_mag_center = torch.sqrt(gradUx[center_idx]**2 + gradUy[center_idx]**2 + gradUz[center_idx]**2)

    p2_center = abs_p2[center_idx]
    p_abs_center = torch.sqrt(p2_center + 1e-32)
    p2_local_rms = torch.sqrt(torch.mean(abs_p2))

    p_pen = pressure_penalty(p2_center, p_abs_center, p2_local_rms)

    w_lap = CURRENT_WEIGHTS['w_lap']
    alpha = CURRENT_WEIGHTS['alpha']
    beta  = CURRENT_WEIGHTS['beta']

    metric = (w_lap * laplacian_center_weighted
              - alpha * grad_mag_center
              - beta  * p_pen)

    return metric, laplacian_center_weighted, grad_mag_center, p_abs_center, laplacian_center_unweighted

def objective_fn_torch(ph_tensor, x_idx, y_idx, z_idx):
    x_local = x_vals[x_idx-half:x_idx+half+1]
    y_local = y_vals[y_idx-half:y_idx+half+1]
    z_local = z_vals[z_idx-half:z_idx+half+1]
    Xl, Yl, Zl = torch.meshgrid(x_local, y_local, z_local, indexing='ij')

    pf_local = compute_pressure_field_torch(ph_tensor, amp_table, wavelength, Xl, Yl, Zl, transducer_positions)
    gorkov_metric, _, _, _, _ = compute_gorkov_objective_local_torch(pf_local, dx, dy, dz)
    return -gorkov_metric

def get_metrics_for_logging_new_method(ph_tensor, x_idx, y_idx, z_idx):
    with torch.no_grad():
        x_local = x_vals[x_idx-half:x_idx+half+1]
        y_local = y_vals[y_idx-half:y_idx+half+1]
        z_local = z_vals[z_idx-half:z_idx+half+1]
        Xl, Yl, Zl = torch.meshgrid(x_local, y_local, z_local, indexing='ij')
        pf_local = compute_pressure_field_torch(ph_tensor, amp_table, wavelength, Xl, Yl, Zl, transducer_positions)

        metric, _, gradmag_c, p_abs_c, true_lap_c = compute_gorkov_objective_local_torch(pf_local, dx, dy, dz)
        loss = -metric
        return float(true_lap_c), float(gradmag_c), float(p_abs_c), float(loss)

# ------------------------------------------------------------
# 슬라이스(단면) — 전체 3D 대신 2D 평면만 계산
# ------------------------------------------------------------
def compute_slice_YZ(phase_vector, x_fixed):
    Yl, Zl = torch.meshgrid(y_vals, z_vals, indexing='ij')
    Xl = torch.full_like(Yl, x_fixed)
    pf = compute_pressure_field_torch(phase_vector, amp_table, wavelength, Xl, Yl, Zl, transducer_positions)
    return torch.abs(pf)

def compute_slice_XZ(phase_vector, y_fixed):
    Xl, Zl = torch.meshgrid(x_vals, z_vals, indexing='ij')
    Yl = torch.full_like(Xl, y_fixed)
    pf = compute_pressure_field_torch(phase_vector, amp_table, wavelength, Xl, Yl, Zl, transducer_positions)
    return torch.abs(pf)

def compute_slice_XY(phase_vector, z_fixed):
    Xl, Yl = torch.meshgrid(x_vals, y_vals, indexing='ij')
    Zl = torch.full_like(Xl, z_fixed)
    pf = compute_pressure_field_torch(phase_vector, amp_table, wavelength, Xl, Yl, Zl, transducer_positions)
    return torch.abs(pf)

# ============================================================
# SciPy 래퍼
# ============================================================
def objective_for_scipy(phases_np):
    phases_torch = torch.tensor(phases_np, dtype=torch.float64, device=device)
    loss = objective_fn_torch(phases_torch, x_idx, y_idx, z_idx)
    return float(loss.item())

def jacobian_for_scipy(phases_np):
    phases_torch = torch.tensor(phases_np, dtype=torch.float64, device=device, requires_grad=True)
    loss = objective_fn_torch(phases_torch, x_idx, y_idx, z_idx)
    (grad,) = torch.autograd.grad(loss, phases_torch, retain_graph=False, create_graph=False)
    return grad.detach().cpu().numpy().astype(np.float64)

# ============================================================
# (간헐) H·v 및 Lanczos 극값 고유치 근사
# ============================================================
def hvp_fd(x_np, v_np, eps=HVP_EPS):
    g_plus = jacobian_for_scipy(x_np + eps*v_np)
    g_minus = jacobian_for_scipy(x_np - eps*v_np)
    return (g_plus - g_minus) / (2.0*eps)

def lanczos_extreme_eigs(hvp_fun, x_np, n, k=10):
    Q = []
    alphas, betas = [], []
    q = np.random.randn(n); q /= (np.linalg.norm(q) + 1e-16)
    beta_prev = 0.0
    for j in range(k):
        v = hvp_fun(x_np, q) if j == 0 else hvp_fun(x_np, q) - beta_prev * Q[-1]
        for qi in Q:
            v -= np.dot(v, qi) * qi
        alpha = float(np.dot(q, v))
        v -= alpha * q
        beta = float(np.linalg.norm(v) + 1e-16)

        Q.append(q.copy())
        alphas.append(alpha); betas.append(beta)
        if beta < 1e-14:
            break
        beta_prev = beta
        q = v / beta

    m = len(alphas)
    if m == 0:
        return np.nan, np.nan
    T = np.zeros((m, m))
    for i in range(m):
        T[i, i] = alphas[i]
        if i+1 < m:
            T[i, i+1] = betas[i+1]
            T[i+1, i] = betas[i+1]
    w = np.linalg.eigvalsh(T)
    return float(w[0]), float(w[-1])

# ============================================================
# 콜백 (스텝 단위 로깅 + 지형 메트릭)
# ============================================================
class StepRecorder:
    def __init__(self, trial_number, trial_start_time, log_list, total_steps, x0_np, optimizer_name):
        self.trial_number = trial_number
        self.trial_start_time = trial_start_time
        self.all_steps_log = log_list
        self.step_counter = 0
        self.total_steps = total_steps
        self.optimizer_name = optimizer_name

        self.n = len(x0_np)
        # 초기 BFGS 상태(Adam/AdamW에서도 의사 p 생성에 사용)
        self.H = np.eye(self.n, dtype=np.float64)
        self.prev_x = x0_np.copy()
        self.prev_loss = objective_for_scipy(self.prev_x)
        self.prev_grad = jacobian_for_scipy(self.prev_x)
        self.prev_p = - self.H.dot(self.prev_grad)

    def __call__(self, xk):
        self.step_counter += 1
        elapsed_time = time.time() - self.trial_start_time

        xk = np.asarray(xk, dtype=np.float64)
        s = xk - self.prev_x
        step_norm = float(np.linalg.norm(s))

        phases_torch = torch.tensor(xk, dtype=torch.float64, device=device)
        true_laplacian, grad_mag, p_abs, loss = get_metrics_for_logging_new_method(
            phases_torch, x_idx, y_idx, z_idx
        )
        curr_loss = float(loss)

        gk = jacobian_for_scipy(xk)
        y = gk - self.prev_grad
        sTy = float(np.dot(s, y))
        valid_curv = (sTy > 1e-16)

        denom_lin = -float(np.dot(self.prev_grad, s))
        actual_reduction = float(self.prev_loss - curr_loss)
        rho_lin = actual_reduction / denom_lin if abs(denom_lin) > 1e-12 else float('nan')

        Hg = self.H.dot(self.prev_grad)
        H2g = self.H.dot(Hg)
        num = float(np.dot(self.prev_grad, Hg))
        den = float(np.dot(self.prev_grad, H2g)) if np.linalg.norm(H2g) > 0 else np.nan
        r_ratio = (num / den) if (den is not np.nan and abs(den) > 1e-300) else np.nan

        p_prev = self.prev_p
        cos_theta = float( np.dot(-self.prev_grad, p_prev) /
                           ((np.linalg.norm(self.prev_grad)+1e-16)*(np.linalg.norm(p_prev)+1e-16)) )

        denom_pp = float(np.dot(p_prev, p_prev))
        alpha_imp = float(np.dot(s, p_prev) / denom_pp) if denom_pp > 1e-300 else np.nan

        cos_phi = float( np.dot(s, y) /
                         ((np.linalg.norm(s)+1e-16)*(np.linalg.norm(y)+1e-16)) ) if np.linalg.norm(y)>0 else np.nan
        gamma_scale = float( np.dot(y, y) / sTy ) if valid_curv else np.nan

        lam_min = lam_max = kappa = np.nan
        if (self.step_counter % SPEC_EVERY == 0) and (self.step_counter > 0):
            try:
                lm, lM = lanczos_extreme_eigs(hvp_fd, xk, self.n, k=SPEC_K)
                lam_min, lam_max = lm, lM
                kappa = float(abs(lam_max) / max(abs(lam_min), 1e-16)) if np.isfinite(lam_min) else np.nan
            except Exception:
                lam_min = lam_max = kappa = np.nan

        row = {
            'optimizer': self.optimizer_name,
            'trial': self.trial_number,
            'step': self.step_counter,
            'time_s': elapsed_time,
            'loss': curr_loss,
            'laplacian': float(true_laplacian),   # 비가중(Uxx+Uyy+Uzz) 기록
            'gorkov_grad_mag': float(grad_mag),
            'pressure_abs': float(p_abs),
            'delta_tr_radius': step_norm,
            'rho_lin': rho_lin,
            'gnorm': float(np.linalg.norm(self.prev_grad)),
            'pnorm': float(np.linalg.norm(p_prev)),
            'alpha_imp': alpha_imp,
            'cos_theta': cos_theta,
            'r_ratio': r_ratio,
            'sTy': sTy,
            'cos_phi': cos_phi,
            'gamma_scale': gamma_scale,
            'lambda_min_est': lam_min,
            'lambda_max_est': lam_max,
            'kappa_est': kappa,
        }
        for i in range(len(xk)):
            row[f'phase_{i}'] = float(xk[i])
        self.all_steps_log.append(row)

        if (self.step_counter % 20 == 0) or (self.step_counter == self.total_steps):
            print(f"  Trial {self.trial_number}, Step {self.step_counter} | "
                  f"loss={curr_loss:.3e} | Δ(step)={step_norm:.3e} | ρ_lin={rho_lin:.3e} | "
                  f"α_imp={alpha_imp:.3e} | cosθ={cos_theta:.3f} | t={elapsed_time:.1f}s")

        # BFGS 업데이트(곡률 양수일 때만)
        if valid_curv:
            rho = 1.0 / sTy
            I = np.eye(self.n, dtype=np.float64)
            I_sy = (I - rho * np.outer(s, y))
            I_ys = (I - rho * np.outer(y, s))
            self.H = I_sy.dot(self.H).dot(I_ys) + rho * np.outer(s, s)

        self.prev_x = xk.copy()
        self.prev_loss = curr_loss
        self.prev_grad = gk.copy()
        self.prev_p = - self.H.dot(self.prev_grad)

# ============================================================
# 시각화 유틸 — 압력장 3단면 (공통 컬러바 상한: XY max)
# ============================================================
def plot_field_slices(yz_abs_np, xz_abs_np, xy_abs_np, trial_no, final_loss):
    x_vals_np = x_vals.cpu().numpy()
    y_vals_np = y_vals.cpu().numpy()
    z_vals_np = z_vals.cpu().numpy()

    common_vmax = float(np.nanmax(xy_abs_np))
    norm = colors.Normalize(vmin=0.0, vmax=common_vmax)

    fig = plt.figure(figsize=(18, 5))
    plt.suptitle(f'Trial {trial_no:03d} | Final Loss: {final_loss:.3e}', fontsize=14)

    ax = plt.subplot(1, 3, 1)
    im1 = ax.imshow(
        yz_abs_np.T,
        extent=[y_vals_np[0], y_vals_np[-1], z_vals_np[0], z_vals_np[-1]],
        origin='lower', aspect='auto', cmap='viridis', norm=norm
    )
    ax.scatter(target_y, target_z, c='red', marker='x', s=60, linewidths=1.5)
    ax.set_title(f'YZ (x={target_x:.3f} m)'); ax.set_xlabel('y (m)'); ax.set_ylabel('z (m)')
    plt.colorbar(im1, ax=ax, label='|p|')

    ax = plt.subplot(1, 3, 2)
    im2 = ax.imshow(
        xz_abs_np.T,
        extent=[x_vals_np[0], x_vals_np[-1], z_vals_np[0], z_vals_np[-1]],
        origin='lower', aspect='auto', cmap='viridis', norm=norm
    )
    ax.scatter(target_x, target_z, c='red', marker='x', s=60, linewidths=1.5)
    ax.set_title(f'XZ (y={target_y:.3f} m)'); ax.set_xlabel('x (m)'); ax.set_ylabel('z (m)')
    plt.colorbar(im2, ax=ax, label='|p|')

    ax = plt.subplot(1, 3, 3)
    im3 = ax.imshow(
        xy_abs_np.T,
        extent=[x_vals_np[0], x_vals_np[-1], y_vals_np[0], y_vals_np[-1]],
        origin='lower', aspect='auto', cmap='viridis', norm=norm
    )
    ax.scatter(target_x, target_y, c='red', marker='x', s=60, linewidths=1.5)
    ax.set_title(f'XY (z={target_z:.3f} m)'); ax.set_xlabel('x (m)'); ax.set_ylabel('y (m)')
    plt.colorbar(im3, ax=ax, label='|p|')

    plt.tight_layout(rect=[0, 0.03, 1, 0.93])
    return fig

# ============================================================
# 결정변수 공간 1D/2D 슬라이스
# ============================================================
def decision_space_slices(objective_fun, x_star, g_star, p_star, trial_id):
    u = -g_star
    if np.linalg.norm(u) > 0:
        u = u / (np.linalg.norm(u)+1e-16)
    else:
        u = np.random.randn(*x_star.shape); u /= np.linalg.norm(u)+1e-16

    v = p_star.copy()
    v = v - np.dot(v, u)*u
    nv = np.linalg.norm(v)
    if nv > 0:
        v = v / nv
    else:
        z = np.random.randn(*x_star.shape)
        z -= np.dot(z, u)*u
        v = z / (np.linalg.norm(v)+1e-16)

    ts = np.linspace(-SLICE_HALF_RANGE, SLICE_HALF_RANGE, SLICE_N_1D)
    f_u = [objective_fun(x_star + t*u) for t in ts]
    f_v = [objective_fun(x_star + t*v) for t in ts]

    A = np.linspace(-SLICE_HALF_RANGE, SLICE_HALF_RANGE, SLICE_N_2D)
    B = np.linspace(-SLICE_HALF_RANGE, SLICE_HALF_RANGE, SLICE_N_2D)
    F = np.zeros((SLICE_N_2D, SLICE_N_2D))
    for i,a in enumerate(A):
        for j,b in enumerate(B):
            F[j,i] = objective_fun(x_star + a*u + b*v)

    fig = plt.figure(figsize=(16,5))
    plt.suptitle(f'Decision-Space Slices (trial {trial_id:03d})', fontsize=16)

    ax1 = plt.subplot(1,3,1)
    ax1.plot(ts, f_u, linewidth=1.5)
    ax1.axvline(0, color='k', linestyle='--', linewidth=0.8)
    ax1.set_title('1D slice along -grad'); ax1.set_xlabel('t (rad)'); ax1.set_ylabel('f(x* + t u)')

    ax2 = plt.subplot(1,3,2)
    ax2.plot(ts, f_v, linewidth=1.5)
    ax2.axvline(0, color='k', linestyle='--', linewidth=0.8)
    ax2.set_title('1D slice along BFGS dir'); ax2.set_xlabel('t (rad)'); ax2.set_ylabel('f(x* + t v)')

    ax3 = plt.subplot(1,3,3)
    im = ax3.imshow(F, extent=[A[0], A[-1], B[0], B[-1]], origin='lower', aspect='auto', cmap='viridis')
    ax3.scatter(0.0, 0.0, c='red', marker='x', s=60, linewidth=2)
    ax3.set_title('2D slice on span{u,v}')
    ax3.set_xlabel('a (rad)'); ax3.set_ylabel('b (rad)')
    plt.colorbar(im, ax=ax3, label='f')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    return fig

# ============================================================
# 메서드 구성 함수 (여기서 메서드별 상수 자동 세팅)
# ============================================================
def configure_method(method_raw: str):
    global METHOD_NAME, PRESSURE_PENALTY_MODE, CURRENT_WEIGHTS, OUTPUT_DIR
    global W_LAPLACIAN, ALPHA_F, BETA_P
    global W_LAPLACIAN_XX, W_LAPLACIAN_YY, W_LAPLACIAN_ZZ

    # 입력 표준화 및 호환 매핑
    method_in = (method_raw or "").strip().lower()
    alias_map = {
        "conventional": "conventional_vortex",
        "regularized_conventional": "regularized_conventional_vortex",
        "regularized_hybrid": "regularized_hybrid_vortex",
        "hybrid": "hybrid_vortex",
    }
    method = alias_map.get(method_in, method_in)

    valid = {
        "conventional_twin",
        "conventional_vortex",
        "regularized_conventional_twin",
        "regularized_conventional_vortex",
        "force-equillibrium",
        "hybrid_twin",
        "hybrid_vortex",
        "regularized_hybrid_twin",
        "regularized_hybrid_vortex",
    }
    if method not in valid:
        raise ValueError(
            f"Unknown method: {method_in}. Choose one of "
            f"{sorted(list(valid | set(alias_map.keys())))}"
        )

    METHOD_NAME = method

    # 공통 기본값
    W_LAPLACIAN = 1.0
    ALPHA_F = 9.0

    # 가중치/패널티 설정
    if method in {
        "conventional_twin", "conventional_vortex",
        "regularized_conventional_twin", "regularized_conventional_vortex"
    }:
        BETA_P = 1.0
        if method in {"conventional_twin", "regularized_conventional_twin"}:
            W_LAPLACIAN_XX, W_LAPLACIAN_YY, W_LAPLACIAN_ZZ = 1000, 10, 10
        elif method in {"conventional_vortex", "regularized_conventional_vortex"}:
            W_LAPLACIAN_XX, W_LAPLACIAN_YY, W_LAPLACIAN_ZZ = 1000, 1000, 10

    elif method in {"force-equillibrium", "hybrid_vortex", "hybrid_twin",
                    "regularized_hybrid_twin", "regularized_hybrid_vortex"}:
        BETA_P = 5e-5
        if method == "hybrid_vortex" or method == "force-equillibrium":
            W_LAPLACIAN_XX = W_LAPLACIAN_YY = W_LAPLACIAN_ZZ = 1
        elif method == "hybrid_twin":
            W_LAPLACIAN_XX, W_LAPLACIAN_YY, W_LAPLACIAN_ZZ = 1, 0.01, 0.01
        elif method == "regularized_hybrid_twin":
            W_LAPLACIAN_XX, W_LAPLACIAN_YY, W_LAPLACIAN_ZZ = 1, 0.01, 0.01
        elif method == "regularized_hybrid_vortex":
            W_LAPLACIAN_XX, W_LAPLACIAN_YY, W_LAPLACIAN_ZZ = 1, 1, 1

    # CURRENT_WEIGHTS & 패널티 모드
    if method in {"conventional_twin", "conventional_vortex"}:
        CURRENT_WEIGHTS = dict(w_lap=W_LAPLACIAN, alpha=0.0, beta=BETA_P)
        PRESSURE_PENALTY_MODE = 'abs'
    elif method in {"regularized_conventional_twin", "regularized_conventional_vortex"}:
        CURRENT_WEIGHTS = dict(w_lap=W_LAPLACIAN, alpha=0.0, beta=BETA_P)
        PRESSURE_PENALTY_MODE = 'smooth_abs'
    elif method == "force-equillibrium":
        CURRENT_WEIGHTS = dict(w_lap=W_LAPLACIAN, alpha=ALPHA_F, beta=0.0)
        PRESSURE_PENALTY_MODE = 'abs'
    elif method == "hybrid_vortex":
        CURRENT_WEIGHTS = dict(w_lap=W_LAPLACIAN, alpha=ALPHA_F, beta=BETA_P)
        PRESSURE_PENALTY_MODE = 'abs'
    elif method == "hybrid_twin":
        CURRENT_WEIGHTS = dict(w_lap=W_LAPLACIAN, alpha=500, beta=BETA_P)
        PRESSURE_PENALTY_MODE = 'abs'
    elif method == "regularized_hybrid_twin":
        CURRENT_WEIGHTS = dict(w_lap=W_LAPLACIAN, alpha=500, beta=BETA_P)
        PRESSURE_PENALTY_MODE = 'smooth_abs'
    elif method == "regularized_hybrid_vortex":
        CURRENT_WEIGHTS = dict(w_lap=W_LAPLACIAN, alpha=ALPHA_F, beta=BETA_P)
        PRESSURE_PENALTY_MODE = 'smooth_abs'

    global OUTPUT_DIR
    OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, METHOD_NAME, RUN_TAG)
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print(f"[INFO] Method: {METHOD_NAME}")
    print(f"[INFO] Weights: {CURRENT_WEIGHTS} | Penalty mode: {PRESSURE_PENALTY_MODE}")
    print(f"[INFO] W_LAPLACIAN_XX,YY,ZZ = {W_LAPLACIAN_XX}, {W_LAPLACIAN_YY}, {W_LAPLACIAN_ZZ}")
    print(f"[INFO] Output dir: {OUTPUT_DIR}")

# ============================================================
# 비교용 옵티마이저 목록 + 옵션
# ============================================================
OPTIMIZERS = ["BFGS", "L-BFGS-B", "Adam", "AdamW"]

PHASE_LO, PHASE_HI = 0.0, 2.0*math.pi

def optimizer_options(name: str):
    if name == "BFGS":
        return {"maxiter": NUM_OPT_STEPS, "gtol": 0, "disp": False}
    if name == "L-BFGS-B":
        return {"maxiter": NUM_OPT_STEPS, "gtol": 0, "disp": False}
    if name == "Adam":
        return {"lr": 1e-2, "weight_decay": 0.0}
    if name == "AdamW":
        return {"lr": 1e-2, "weight_decay": 1e-4}
    return {"maxiter": NUM_OPT_STEPS, "disp": False}

# ---- Torch optimizer runner (Adam/AdamW) ----
def run_torch_optimizer(opt_name: str, x0_np: np.ndarray, recorder: StepRecorder):
    x = torch.tensor(x0_np, dtype=torch.float64, device=device, requires_grad=True)
    opts = optimizer_options(opt_name)
    if opt_name == "Adam":
        optim = torch.optim.Adam([x], lr=opts["lr"], weight_decay=opts["weight_decay"])
    elif opt_name == "AdamW":
        optim = torch.optim.AdamW([x], lr=opts["lr"], weight_decay=opts["weight_decay"])
    else:
        raise ValueError(f"Unsupported torch optimizer: {opt_name}")

    best_x = None
    best_f = float("inf")

    for _ in range(1, NUM_OPT_STEPS + 1):
        optim.zero_grad()
        loss = objective_fn_torch(x, x_idx, y_idx, z_idx)
        loss.backward()
        optim.step()
        with torch.no_grad():
            x.data = (x.data % (2.0 * math.pi))
        recorder(np.ascontiguousarray(x.detach().cpu().numpy(), dtype=np.float64))
        f = float(loss.item())
        if f < best_f:
            best_f = f
            best_x = x.detach().cpu().numpy().copy()

    return {"x": best_x if best_x is not None else x.detach().cpu().numpy(),
            "fun": best_f if best_x is not None else float(loss.item()),
            "nit": NUM_OPT_STEPS}

# ============================================================
# 메인 루프
# ============================================================
def main():
    all_steps_log = []

    print(f"\n[INFO] Starting {NUM_TRIALS} trials per optimizer, max iter={NUM_OPT_STEPS}")
    total_start_time = time.time()

    for opt_name in OPTIMIZERS:
        print(f"\n================ OPTIMIZER: {opt_name} ================")

        bounds = [(PHASE_LO, PHASE_HI)] * num_transducers if opt_name == "L-BFGS-B" else None
        opt_out_dir = os.path.join(OUTPUT_DIR, f"optimizer_{opt_name}")
        os.makedirs(opt_out_dir, exist_ok=True)

        for i in range(NUM_TRIALS):
            trial_start_time = time.time()
            trial_no = i + 1
            print(f"\n--- [{opt_name}] Trial {trial_no}/{NUM_TRIALS} ---")

            initial_phases_np = np.random.rand(num_transducers) * 2.0 * math.pi

            recorder = StepRecorder(
                trial_number=trial_no,
                trial_start_time=trial_start_time,
                log_list=all_steps_log,
                total_steps=NUM_OPT_STEPS,
                x0_np=initial_phases_np,
                optimizer_name=opt_name
            )

            if opt_name in ("Adam", "AdamW"):
                res = run_torch_optimizer(opt_name, initial_phases_np, recorder)
            else:
                res = minimize(
                    fun=objective_for_scipy,
                    x0=initial_phases_np,
                    method=opt_name,
                    jac=jacobian_for_scipy,
                    callback=recorder,
                    bounds=bounds,
                    options=optimizer_options(opt_name)
                )

            trial_total_time = time.time() - trial_start_time
            final_fun = res["fun"] if isinstance(res, dict) else res.fun
            final_x   = res["x"]   if isinstance(res, dict) else res.x
            final_nit = res["nit"] if isinstance(res, dict) else getattr(res, "nit", -1)
            print(f"[INFO] [{opt_name}] Trial {trial_no} Done | Elapsed {trial_total_time:.2f}s | "
                  f"Final loss {final_fun:.3e} | iters={final_nit}")

            if GENERATE_SLICE_PLOTS:
                print("  [INFO] Computing ONLY 3 slices (YZ/XZ/XY) and saving plots...")
                with torch.no_grad():
                    final_phases = torch.tensor(final_x, dtype=torch.float64, device=device)
                    yz_abs = compute_slice_YZ(final_phases, x_vals[x_idx])
                    xz_abs = compute_slice_XZ(final_phases, y_vals[y_idx])
                    xy_abs = compute_slice_XY(final_phases, z_vals[z_idx])

                    fig = plot_field_slices(yz_abs.cpu().numpy(), xz_abs.cpu().numpy(), xy_abs.cpu().numpy(),
                                            trial_no, final_fun)
                    filename = os.path.join(opt_out_dir, f'trial_{trial_no:03d}_field_slices.png')
                    fig.savefig(filename, dpi=150); plt.close(fig)
                    print(f"  [INFO] Plot saved: {filename}")

            if GENERATE_DECISION_SLICES:
                print("  [INFO] Computing decision-space slices at final point ...")
                x_final = np.asarray(final_x, dtype=np.float64)
                g_final = jacobian_for_scipy(x_final)
                try:
                    p_final = - recorder.H.dot(g_final)
                except Exception:
                    p_final = - g_final

                fig2 = decision_space_slices(objective_for_scipy, x_final, g_final, p_final, trial_no)
                filename2 = os.path.join(opt_out_dir, f"trial_{trial_no:03d}_decision_slices.png")
                fig2.savefig(filename2, dpi=170); plt.close(fig2)
                print(f"  [INFO] Decision-space slices saved: {filename2}")

            if device.type == 'cuda':
                torch.cuda.empty_cache()

    total_time = time.time() - total_start_time
    print(f"\n[INFO] Total execution time for {NUM_TRIALS} trials: {total_time:.2f} s")

    # --------------------------------------------------------
    # CSV 저장 (매 스텝 phase + 지형 메트릭 포함)
    # --------------------------------------------------------
    print("\n[INFO] Creating & saving per-step CSV ...")
    df_final = pd.DataFrame(all_steps_log)

    metric_cols = [
        'step','time_s','loss','laplacian','gorkov_grad_mag','pressure_abs',
        'delta_tr_radius','rho_lin',
        'gnorm','pnorm','alpha_imp','cos_theta','r_ratio','sTy','cos_phi','gamma_scale',
        'lambda_min_est','lambda_max_est','kappa_est'
    ]
    phase_cols = [f'phase_{i}' for i in range(num_transducers)]
    final_cols = ['optimizer','trial'] + metric_cols + phase_cols
    df_final = df_final[final_cols]

    csv_path = os.path.join(OUTPUT_DIR, "all_optimizers_steps_with_phases.csv")
    df_final.to_csv(csv_path, index=False)
    print(f"[OK] Saved CSV with {len(df_final)} rows: {csv_path}")

    # ===== 수렴속도(median/IQR) + 중앙값/IQR 리포트 추가 =====
    CONV_TARGET_LOSS = None  # 필요시 절대값 지정, 예:  -1.0e5  (없으면 전체 중앙값 사용)

    # 최종 스텝(각 trial 마지막) 집계
    last_rows = (df_final.sort_values(['optimizer','trial','step'])
                          .groupby(['optimizer','trial'], as_index=False)
                          .tail(1))

    # 기준 손실 결정
    target_loss = CONV_TARGET_LOSS if CONV_TARGET_LOSS is not None else last_rows['loss'].median()
    print(f"\n[INFO] Convergence target loss = {target_loss:.6e}")

    def iqr(s):
        return float(s.quantile(0.75) - s.quantile(0.25))

    summary_records = []
    for opt_name in df_final['optimizer'].unique():
        dfo = df_final[df_final['optimizer']==opt_name].copy()

        # 각 trial별 '처음으로 target 이하' 도달 시점 찾기
        conv_list = []
        for t, dft in dfo.groupby('trial'):
            dft = dft.sort_values('step')
            hit = dft[dft['loss'] <= target_loss]
            if len(hit) > 0:
                first = hit.iloc[0]
                conv_list.append({
                    'optimizer': opt_name,
                    'trial': int(t),
                    'steps_to_target': int(first['step']),
                    'time_to_target': float(first['time_s'])
                })
        df_conv = pd.DataFrame(conv_list)

        # 최종 스텝의 통계(손실, step 시간 등)
        finals = last_rows[last_rows['optimizer']==opt_name]
        rec = {
            'optimizer': opt_name,
            # 최종 손실 분포
            'final_loss_median': float(finals['loss'].median()),
            'final_loss_IQR':    float(iqr(finals['loss'])),
            # 최종 압력/그라디언트 참고
            'final_pressure_median': float(finals['pressure_abs'].median()),
            'final_pressure_IQR':    float(iqr(finals['pressure_abs'])),
            # 스텝당 시간 분포(마지막 스텝의 누적시간으로 근사)
            'final_time_median_s': float(finals['time_s'].median()),
            'final_time_IQR_s':    float(iqr(finals['time_s'])),
            # 수렴속도(타깃 도달) 분포
            'steps_to_target_median': float(df_conv['steps_to_target'].median()) if not df_conv.empty else float('nan'),
            'steps_to_target_IQR':    float(iqr(df_conv['steps_to_target']))     if not df_conv.empty else float('nan'),
            'time_to_target_median_s': float(df_conv['time_to_target'].median()) if not df_conv.empty else float('nan'),
            'time_to_target_IQR_s':    float(iqr(df_conv['time_to_target']))     if not df_conv.empty else float('nan'),
            # 타깃 도달율
            'reach_rate': float(len(df_conv)) / finals['trial'].nunique() if finals['trial'].nunique()>0 else float('nan')
        }
        summary_records.append(rec)

    df_summary_all = pd.DataFrame(summary_records)
    sum_path = os.path.join(OUTPUT_DIR, "summary_per_optimizer.csv")
    df_summary_all.to_csv(sum_path, index=False)
    print("\n===== Convergence & Distribution (median/IQR) =====")
    print(df_summary_all.to_string(index=False))
    print(f"[OK] Saved per-optimizer summary: {sum_path}")

    # 요약: step==NUM_OPT_STEPS 우선, 없으면 각 trial의 마지막 스텝
    print("\n" + "="*64)
    print(" Per-Optimizer Summary (last step per trial fallback)")
    print("="*64)
    if 'step' in df_final.columns:
        for opt_name in OPTIMIZERS:
            df_opt = df_final[df_final['optimizer']==opt_name]
            df_summary = df_opt[df_opt['step'] == NUM_OPT_STEPS]
            if df_summary.empty:
                df_summary = (df_opt.sort_values(['trial','step'])
                                   .groupby('trial', as_index=False)
                                   .tail(1))
            if df_summary.empty:
                print(f"[WARN] No rows for {opt_name}"); continue
            print(f"\n[{opt_name}] Trials summarized: {df_summary['trial'].nunique()}")
            for col in ['loss','laplacian','pressure_abs','gorkov_grad_mag',
                        'delta_tr_radius','rho_lin','alpha_imp','cos_theta']:
                mu = df_summary[col].mean(); sd = df_summary[col].std(); mn = df_summary[col].min()
                print(f"{col:15s} | Mean: {mu:.4e} | Std: {sd:.4e} | Min: {mn:.4e}")
            print(f"Step Time (s)   | Mean: {df_summary['time_s'].mean():.2f} | "
                  f"Std: {df_summary['time_s'].std():.2f}")
    else:
        print("No 'step' column in final dataframe.")
    print("="*64)

if __name__ == "__main__":
    prompt = ("Select method [conventional_twin | conventional_vortex | "
              "regularized_conventional_twin | regularized_conventionazl_vortex | "
              "force-equillibrium | hybrid_twin | hybrid_vortex | "
              "regularized_hybrid_twin | regularized_hybrid_vortex]\n"
              "(compat: 'conventional'->conventional_vortex, "
              "'regularized_conventional'->regularized_conventional_vortex, "
              "'regularized_hybrid'->regularized_hybrid_vortex, "
              "'hybrid'->hybrid_vortex): ")
    user_method = input(prompt)
    configure_method(user_method)
    main()


[INFO] Using device: cpu
[INFO] Using Fixed Random Seed: 42
[INFO] Transducer grid: 8x8 (64 transducers).
[INFO] Target Index: (120, 120, 60) | Target Coord: (0.0100, 0.0100, 0.0300)
Select method [conventional_twin | conventional_vortex | regularized_conventional_twin | regularized_conventionazl_vortex | force-equillibrium | hybrid_twin | hybrid_vortex | regularized_hybrid_twin | regularized_hybrid_vortex]
(compat: 'conventional'->conventional_vortex, 'regularized_conventional'->regularized_conventional_vortex, 'regularized_hybrid'->regularized_hybrid_vortex, 'hybrid'->hybrid_vortex): regularized_hybrid
[INFO] Method: regularized_hybrid_vortex
[INFO] Weights: {'w_lap': 1.0, 'alpha': 9.0, 'beta': 5e-05} | Penalty mode: smooth_abs
[INFO] W_LAPLACIAN_XX,YY,ZZ = 1, 1, 1
[INFO] Output dir: results_github/regularized_hybrid_vortex/BFGS_8x8_rand42_fixed_10000_steps

[INFO] Starting 100 trials per optimizer, max iter=10000

================ OPTIMIZER: BFGS ================

--- [BFGS] Trial 1

/tmp/ipython-input-1457862507.py:734: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  res = minimize(


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  Trial 91, Step 2660 | loss=-6.770e+00 | Δ(step)=1.529e-03 | ρ_lin=5.486e-01 | α_imp=4.308e-14 | cosθ=0.000 | t=59.6s
  Trial 91, Step 2680 | loss=-6.770e+00 | Δ(step)=2.387e-03 | ρ_lin=3.884e+00 | α_imp=5.729e-16 | cosθ=0.000 | t=60.0s
  Trial 91, Step 2700 | loss=-6.770e+00 | Δ(step)=5.037e-03 | ρ_lin=-4.204e-01 | α_imp=2.319e-15 | cosθ=0.000 | t=60.5s
  Trial 91, Step 2720 | loss=-6.770e+00 | Δ(step)=1.161e-03 | ρ_lin=7.320e+00 | α_imp=-4.216e-16 | cosθ=0.000 | t=60.9s
  Trial 91, Step 2740 | loss=-6.770e+00 | Δ(step)=8.234e-03 | ρ_lin=2.199e-01 | α_imp=2.957e-16 | cosθ=0.000 | t=61.3s
  Trial 91, Step 2760 | loss=-6.769e+00 | Δ(step)=4.527e-03 | ρ_lin=9.656e-01 | α_imp=2.145e-17 | cosθ=0.000 | t=61.8s
  Trial 91, Step 2780 | loss=-6.769e+00 | Δ(step)=3.961e-03 | ρ_lin=1.199e+00 | α_imp=-8.415e-15 | cosθ=0.000 | t=62.3s
  Trial 91, Step 2800 | loss=-6.769e+00 | Δ(step)=8.071e-03 | ρ_lin=-5.956e-01 | α_imp=2.613e-16 | cosθ=0.000 | t=62.7s
  Trial 

In [ ]:
# 📦 특정 폴더 압축 후 다운로드

from google.colab import files
import shutil
import os

# 🔹 압축할 폴더 경로 지정
folder_path = "/content/results_github"  # 여기를 원하는 폴더 경로로 변경

# 🔹 압축 파일 이름 지정
zip_name = "results_github.zip"

# 🔹 기존 zip이 있다면 삭제
if os.path.exists(zip_name):
    os.remove(zip_name)

# 🔹 폴더를 zip으로 압축
shutil.make_archive(zip_name.replace(".zip", ""), 'zip', folder_path)

# 🔹 압축 파일 다운로드
files.download(zip_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Colab: unzip(한글/특이이름 대응) + 폴더 자동탐색 + 통계검정
!pip -q install statsmodels

from google.colab import drive
drive.mount('/content/drive')

import os, glob, zipfile, math
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon, fisher_exact
from statsmodels.stats.multitest import multipletests

# -------------------------------------------------------------------
# 0) 기본 경로 (드라이브의 'experiments' 폴더)
# -------------------------------------------------------------------
EXP_ROOT = "/content/drive/MyDrive/experiments"
os.makedirs(EXP_ROOT, exist_ok=True)

# -------------------------------------------------------------------
# 1) zip(혹은 이름이 이상한 압축파일) → 풀어서 conventional / regularized_hybrid에 배치
#    - 파일명이 '...의 사본'이어도 동작
#    - 이미 폴더가 있으면 유지
# -------------------------------------------------------------------
def safe_unzip(src, dest):
    os.makedirs(dest, exist_ok=True)
    try:
        with zipfile.ZipFile(src, 'r') as zf:
            zf.extractall(dest)
            print(f"[OK] Unzipped: {src} -> {dest}")
    except zipfile.BadZipFile:
        print(f"[WARN] Not a valid zip (skip): {src}")

# 후보 압축 파일 수집(확장자 무시, 이름으로 분류)
cands = glob.glob(os.path.join(EXP_ROOT, "*"))
for src in cands:
    if os.path.isdir(src):
        continue
    name = os.path.basename(src).lower()
    if "conventional" in name:
        safe_unzip(src, os.path.join(EXP_ROOT, "conventional"))
    elif "regularized_hybrid" in name:
        safe_unzip(src, os.path.join(EXP_ROOT, "regularized_hybrid"))

# -------------------------------------------------------------------
# 2) 큰 CSV가 들어있는 "실험 폴더" 자동 탐색
# -------------------------------------------------------------------
def find_candidate_dirs(root_dir):
    pat = os.path.join(root_dir, "**", "all_optimizers_steps_with_phases.csv")
    return sorted(set(os.path.dirname(p) for p in glob.glob(pat, recursive=True)))

conv_dirs = find_candidate_dirs(os.path.join(EXP_ROOT, "conventional"))
reg_dirs  = find_candidate_dirs(os.path.join(EXP_ROOT, "regularized_hybrid"))

print("\n[SCAN] conventional candidates:\n ", "\n  ".join(conv_dirs) if conv_dirs else "(none)")
print("[SCAN] regularized_hybrid candidates:\n ", "\n  ".join(reg_dirs) if reg_dirs else "(none)")

BASE_DIRS = []
if conv_dirs: BASE_DIRS.append(conv_dirs[0])
if reg_dirs:  BASE_DIRS.append(reg_dirs[0])
print("\nBASE_DIRS =", BASE_DIRS)
assert BASE_DIRS, "분석할 폴더를 찾지 못했습니다. 위 목록을 보고 가장 안쪽 폴더 경로를 직접 지정하세요."

# -------------------------------------------------------------------
# 3) 유틸: IQR 분리 여부 + Wilcoxon(쌍대) + Holm + Cliff's δ(paired)
# -------------------------------------------------------------------
def iqr_band(m,i): return (m - i/2.0, m + i/2.0)
def iqr_separated(m1,i1,m2,i2):
    a1, a2 = iqr_band(m1,i1), iqr_band(m2,i2)
    return (a1[1] < a2[0]) or (a2[1] < a1[0])

def paired_wilcoxon_with_cliffs(a, b):
    a = np.asarray(a); b = np.asarray(b)
    mask = np.isfinite(a) & np.isfinite(b)
    a, b = a[mask], b[mask]
    n = len(a)
    if n == 0: return np.nan, np.nan, 0
    stat, p = wilcoxon(a, b, zero_method="wilcox", alternative="two-sided", correction=False)
    d = a - b
    delta = ((d > 0).sum() - (d < 0).sum()) / float(n)  # [-1,1]
    return float(p), float(delta), int(n)

# -------------------------------------------------------------------
# 4) 폴더별 처리: per_trial_summary 생성(스트리밍) + 검정
# -------------------------------------------------------------------
CHUNKSIZE = 500_000

def process_dir(D):
    big  = os.path.join(D, "all_optimizers_steps_with_phases.csv")
    summ = os.path.join(D, "summary_per_optimizer.csv")
    print("\n" + "="*100); print("[DIR]", D)
    if not os.path.exists(big):
        print("  [SKIP] big CSV 없음:", big); return

    # (A) 요약 파일이 있으면 IQR 분리 여부 빠른 확인
    if os.path.exists(summ):
        try:
            df_sum = pd.read_csv(summ)
            def row(opt): return df_sum.loc[df_sum["optimizer"]==opt].iloc[0]
            for med_col, iqr_col, label in [
                ("time_to_target_median_s","time_to_target_IQR_s","time_to_target"),
                ("steps_to_target_median","steps_to_target_IQR","steps_to_target"),
            ]:
                if med_col in df_sum.columns and iqr_col in df_sum.columns:
                    rB, rA, rW = row("BFGS"), row("Adam"), row("AdamW")
                    sepA = iqr_separated(rB[med_col], rB[iqr_col], rA[med_col], rA[iqr_col])
                    sepW = iqr_separated(rB[med_col], rB[iqr_col], rW[med_col], rW[iqr_col])
                    dA = float(rA[med_col] - rB[med_col]); dW = float(rW[med_col] - rB[med_col])
                    print(f"  [IQR] {label}: Δmedian(Adam-BFGS)={dA:.3f}, sep={sepA} | "
                          f"Δmedian(AdamW-BFGS)={dW:.3f}, sep={sepW}")
        except Exception as e:
            print("  [IQR] skip:", e)

    # (B) TARGET_LOSS 결정(요약 있으면 그 값, 없으면 마지막 loss의 BFGS 중앙값)
    TARGET_LOSS = None
    if os.path.exists(summ):
        try:
            df_sum = pd.read_csv(summ)
            TARGET_LOSS = float(df_sum.loc[df_sum["optimizer"]=="BFGS", "final_loss_median"].iloc[0])
            print(f"  [TARGET] from summary (BFGS final_loss_median): {TARGET_LOSS:.6e}")
        except Exception:
            TARGET_LOSS = None

    if TARGET_LOSS is None:
        finals = []
        use = ["optimizer","trial","step","loss"]
        for ch in pd.read_csv(big, chunksize=CHUNKSIZE, usecols=use):
            last = (ch.sort_values(use)
                      .groupby(["optimizer","trial"], as_index=False)
                      .tail(1))
            finals.append(last[["optimizer","trial","loss"]])
        finals = pd.concat(finals, ignore_index=True)
        TARGET_LOSS = finals.loc[finals["optimizer"]=="BFGS", "loss"].median()
        print(f"  [TARGET] from last rows (BFGS median): {TARGET_LOSS:.6e}")

    # (C) 스트리밍 요약(per_trial_summary.csv)
    hit_time, hit_step, last_time, last_step, final_loss = {}, {}, {}, {}, {}
    use = ["optimizer","trial","step","time_s","loss"]
    for ch in pd.read_csv(big, chunksize=CHUNKSIZE, usecols=use):
        ch.sort_values(["optimizer","trial","step"], inplace=True)
        for (opt, trial), g in ch.groupby(["optimizer","trial"], sort=False):
            key = (opt, int(trial))
            last = g.iloc[-1]; s = int(last["step"])
            if (key not in last_step) or (s > last_step[key]):
                last_step[key] = s
                last_time[key] = float(last["time_s"])
                final_loss[key] = float(last["loss"])
            hit = g[g["loss"] <= TARGET_LOSS]
            if len(hit) > 0:
                first = hit.iloc[0]; fs = int(first["step"])
                if (key not in hit_step) or (fs < hit_step[key]):
                    hit_step[key] = fs; hit_time[key] = float(first["time_s"])

    rows = []
    for opt, trial in sorted(set(last_time.keys()) | set(hit_time.keys())):
        rows.append({
            "optimizer": opt, "trial": trial,
            "reached": int((opt,trial) in hit_time),
            "time_to_target": hit_time.get((opt,trial), np.nan),
            "steps_to_target": hit_step.get((opt,trial), np.nan),
            "final_time": last_time.get((opt,trial), np.nan),
            "final_loss": final_loss.get((opt,trial), np.nan),
        })
    df = pd.DataFrame(rows)
    out_csv = os.path.join(D, "per_trial_summary.csv")
    df.to_csv(out_csv, index=False)
    print(f"  [OK] wrote {out_csv}  shape={df.shape}")

    # (D) 통계검정
    print("  [TESTS]")
    def reach_counts(opt):
        sub = df[df["optimizer"]==opt]
        return int(sub["reached"].sum()), int(len(sub) - sub["reached"].sum())

    # Fisher: reach (BFGS vs L-BFGS-B)
    try:
        bsucc,bfail = reach_counts("BFGS")
        lsucc,lfail = reach_counts("L-BFGS-B")
        _, p_f = fisher_exact([[bsucc,bfail],[lsucc,lfail]])
        print(f"    Fisher (reach) BFGS vs L-BFGS-B: p={p_f:.2e} | counts=({bsucc},{bfail}) vs ({lsucc},{lfail})")
    except Exception as e:
        print("    Fisher skipped:", e)

    # Paired Wilcoxon (성공 trial 교집합)
    def paired_series(metric, other):
        piv = df.pivot(index="trial", columns="optimizer", values=[metric,"reached"])
        mask = (piv[("reached","BFGS")]==1) & (piv[("reached",other)]==1)
        a = piv[(metric,"BFGS")][mask].dropna()
        b = piv[(metric,other)][mask].loc[a.index].dropna()
        return a.values, b.values

    tests = []
    for other in ["Adam","AdamW"]:
        for metric in ["time_to_target","steps_to_target"]:
            try:
                a, b = paired_series(metric, other)
                p, d, n = paired_wilcoxon_with_cliffs(a, b)
                if not math.isnan(p) and n>0:
                    tests.append({"contrast": f"BFGS vs {other}", "metric": metric,
                                  "n": n, "p_raw": p, "cliffs_delta": d})
            except Exception:
                pass
    df_tests = pd.DataFrame(tests)
    if not df_tests.empty:
        df_tests["p_holm"] = multipletests(df_tests["p_raw"], method="holm")[1]
        print("    Paired Wilcoxon + Holm")
        print(df_tests.to_string(index=False))
        for _,r in df_tests.iterrows():
            print(f"      -> {r['contrast']} {r['metric']}: n={int(r['n'])}, "
                  f"p(Holm)={r['p_holm']:.2e}, Cliff's δ(paired)={r['cliffs_delta']:.2f}")
    else:
        print("    Paired Wilcoxon: no overlapping successful trials.")

# -------------------------------------------------------------------
# 5) 실행
# -------------------------------------------------------------------
for d in BASE_DIRS:
    process_dir(d)

print("\n[Done] 모든 폴더 처리 완료.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[OK] Unzipped: /content/drive/MyDrive/experiments/regularized_hybrid.zip의 사본 -> /content/drive/MyDrive/experiments/regularized_hybrid
[OK] Unzipped: /content/drive/MyDrive/experiments/conventional.zip의 사본 -> /content/drive/MyDrive/experiments/conventional

[SCAN] conventional candidates:
  /content/drive/MyDrive/experiments/conventional/conventional_vortex/BFGS_8x8_rand42_fixed_10000_steps
[SCAN] regularized_hybrid candidates:
  /content/drive/MyDrive/experiments/regularized_hybrid/regularized_hybrid_vortex/BFGS_8x8_rand42_fixed_10000_steps

BASE_DIRS = ['/content/drive/MyDrive/experiments/conventional/conventional_vortex/BFGS_8x8_rand42_fixed_10000_steps', '/content/drive/MyDrive/experiments/regularized_hybrid/regularized_hybrid_vortex/BFGS_8x8_rand42_fixed_10000_steps']

[DIR] /content/drive/MyDrive/experiments/conventional/conventional_vortex/BFGS_

In [ ]:
# === ECI–NTS scatter for 4 optimizers × 2 objectives ===
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# 1) 두 objective의 summary_per_optimizer.csv 자동 탐색
root = Path('/content/drive/MyDrive/experiments')
cands = list(root.rglob('summary_per_optimizer.csv'))
if not cands:
    raise FileNotFoundError("summary_per_optimizer.csv 파일을 /content/drive/MyDrive/experiments 아래에서 찾지 못했습니다.")

# objective 이름을 경로에서 추론(폴더명에 'conventional' 또는 'regularized_hybrid'가 포함됨)
def infer_objective(p: Path):
    s = str(p).lower()
    if 'conventional' in s:
        return 'conventional'
    if 'regularized_hybrid' in s:
        return 'regularized_hybrid'
    return 'unknown'

dfs = []
for p in cands:
    obj = infer_objective(p)
    try:
        df = pd.read_csv(p)
        df['objective'] = obj
        df['src'] = str(p)
        dfs.append(df)
    except Exception as e:
        print(f"[WARN] {p}: {e}")

df = pd.concat(dfs, ignore_index=True)
# 필요한 열이름 표준화(코드 기반 요약과 일치)
# 기대 열: optimizer, final_time_median_s, time_to_target_median_s, reach_rate, steps_to_target_median
if 'time_to_target_median_s' not in df.columns:
    # 일부 파일명 다르면 추정: 'time_to_target_median' 등
    for alt in ['time_to_target_median', 'time_to_target_median_sec']:
        if alt in df.columns:
            df['time_to_target_median_s'] = df[alt]; break
if 'final_time_median_s' not in df.columns:
    for alt in ['final_time_median', 'final_time_median_sec', 'final_time_median_s ']:
        if alt in df.columns:
            df['final_time_median_s'] = df[alt]; break
if 'steps_to_target_median' not in df.columns:
    for alt in ['median_steps_to_target', 'steps_median']:
        if alt in df.columns:
            df['steps_to_target_median'] = df[alt]; break

# 2) 8개 조합만 필터(optimizer 4종)
OPTIMIZERS = ['BFGS','L-BFGS-B','Adam','AdamW']
df = df[df['optimizer'].isin(OPTIMIZERS)].copy()

# 3) ECI = reach_rate (없으면 0으로 대체)
df['ECI'] = pd.to_numeric(df.get('reach_rate', np.nan), errors='coerce').fillna(0.0)

# 4) NTS 계산: 같은 objective 내 BFGS의 time_to_target_median_s를 baseline으로
def pick_time(row):
    t = row.get('time_to_target_median_s', np.nan)
    if pd.isna(t):
        t = row.get('final_time_median_s', np.nan)
    return t

df['time_pick_s'] = df.apply(pick_time, axis=1)

nts_list = []
for obj, g in df.groupby('objective'):
    # BFGS baseline
    base = g.loc[g['optimizer']=='BFGS', 'time_to_target_median_s'].dropna()
    if base.empty:
        # 희귀 케이스: BFGS가 time_to_target NaN이면 final_time 사용
        base = g.loc[g['optimizer']=='BFGS', 'time_pick_s'].dropna()
    baseline = float(base.iloc[0]) if not base.empty else np.nan
    # NTS = time_pick / baseline
    tmp = g.copy()
    tmp['NTS'] = tmp['time_pick_s'] / baseline if np.isfinite(baseline) else np.nan
    nts_list.append(tmp)

df_nts = pd.concat(nts_list, ignore_index=True)

# 5) 산점도 (x=NTS, y=ECI)
markers = {'BFGS':'o','L-BFGS-B':'X','Adam':'s','AdamW':'^'}
colors = {'conventional':'tab:blue','regularized_hybrid':'tab:orange','unknown':'gray'}

plt.figure(figsize=(7,5))
for _, r in df_nts.iterrows():
    x, y = r['NTS'], r['ECI']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<Figure size 700x500 with 0 Axes>